In [1]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys

import core.data as data

logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [2]:
import braindecode
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# load nn model from braindecode
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from braindecode.models.deep4 import Deep4Net
from torch import nn
from braindecode.torch_ext.util import set_random_seeds

import core.data as data

%load_ext autoreload
%autoreload 2

In [3]:
# load data 
x_loaded, y_loaded = data.load_x('data/x_train.h5'), data.load_y('data/y_train.csv')

Started loading file data/x_train.h5
Finished loading the file.
Started loading file data/y_train.csv
Finished loading the file.


In [4]:
# convert y to categorical with np.eye(d)[y_loaded]
# and flatten the 40 independent samples
x, y = data.flatten_x(x_loaded), data.flatten_y(y_loaded, repeat=40)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.66)
x_train, x_test = x_train.squeeze(), x_test.squeeze()

# Only one value : 0 or 1  
y_train, y_test = np.argmax(y_train, axis=1), np.argmax(y_test, axis=1)

# get class weights
weights = data.class_weights(y_train)

print('shape : ', x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print('weights :', weights)

shape :  (24974, 7, 500) (12866, 7, 500) (24974,) (12866,)
weights : [0.64276522 2.25112674]


In [5]:
# define the model

cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = x_train.shape[1]
input_time_length=x_train.shape[2]
final_conv_length='auto'
# final_conv_length = auto ensures we only get a single output in the time dimension
# model = Deep4Net(in_chans=in_chans, n_classes=n_classes, input_time_length=input_time_length, final_conv_length=final_conv_length)
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=input_time_length,
                        final_conv_length=final_conv_length)

from braindecode.torch_ext.optimizers import AdamW
import torch.nn.functional as F
#optimizer = AdamW(model.parameters(), lr=1*0.01, weight_decay=0.5*0.001) # these are good values for the deep model
optimizer = AdamW(model.parameters(), lr=0.0625 * 0.01, weight_decay=0)
criterion = lambda prediction, targets : F.nll_loss(prediction, targets, weight=torch.from_numpy(weights).float())
model.compile(loss=criterion, optimizer=optimizer, iterator_seed=1,)

In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=64, scheduler='cosine'
         ,validation_data=(x_test, y_test)
         ,input_time_length = 450) # supercropsize for cropped training
# Rk : here, 1 timestep = 1 / 250 seconds

2020-03-19 11:34:12,115 INFO : Run until first stop...
2020-03-19 11:36:05,460 INFO : Epoch 0
2020-03-19 11:36:05,469 INFO : train_loss                1.12878
2020-03-19 11:36:05,472 INFO : valid_loss                1.13823
2020-03-19 11:36:05,475 INFO : train_misclass            0.53372
2020-03-19 11:36:05,478 INFO : valid_misclass            0.53093
2020-03-19 11:36:05,480 INFO : runtime                   0.00000
2020-03-19 11:36:05,482 INFO : 
2020-03-19 11:45:12,965 INFO : Time only for training updates: 547.48s
2020-03-19 11:46:56,014 INFO : Epoch 1
2020-03-19 11:46:56,020 INFO : train_loss                1.06002
2020-03-19 11:46:56,021 INFO : valid_loss                1.06488
2020-03-19 11:46:56,021 INFO : train_misclass            0.76167
2020-03-19 11:46:56,031 INFO : valid_misclass            0.76325
2020-03-19 11:46:56,036 INFO : runtime                   660.84841
2020-03-19 11:46:56,037 INFO : 
2020-03-19 11:56:03,210 INFO : Time only for training updates: 547.17s
2020-03-1

In [ ]:
x_challenge = data.load_x('data/x_test.h5')
x_challenge = data.flatten_x(x_challenge).squeeze()

In [ ]:
# average the predictions of all the indepedent trials to return one classification for each subject
def average_predictions(predictions, nb_trials = 40):
    # number of samples
    n = int(len(predictions) / nb_trials)
    avg_preds = np.zeros(n)
    for i in range(n):
        sample_preds = predictions[i*nb_trials:(i+1)*nb_trials]
        avg_preds[i] = int(np.mean(sample_preds) > 0.5)
    return avg_preds

In [ ]:
y_challenge = model.predict_classes(x_challenge)
y_challenge2 = data.average_predictions(y_challenge)

In [ ]:
from core.data import save_csv 
save_csv(y_challenge2, 'data/result.csv')

### Vizualise the neural network (recepive field)